In [1]:
import os
import json
from collections import deque
import numpy as np
import threading
import signal 
import time
import matplotlib; matplotlib.use('TkAgg') 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys

sys.path.append("/home/pavnet/processing2/dataprocessingv2/code/")
# pavnet's modules
import utils
import station

In [4]:
curr_dir = os.getcwd()#os.path.dirname(__file__)
with open(f'{curr_dir}/config_params.json', 'r') as f:
    config = json.load(f)
    #print(json.dumps(config, indent=4))

dspconfig = config["DSP"]
sproc = station.dsp(dspconfig)
Txs = dspconfig["vlf_transmitters"].keys()
backup_path = None #"/data/pavnet/datafiles/"
ramdisk = "/run/user/1000/temprpdata/"
SLEEPT = 2 #seconds

[0.00000000e+00 1.22070312e+01 2.44140625e+01 ... 9.99633789e+04
 9.99755859e+04 9.99877930e+04]
DSP Set Ok


In [7]:
path = "/media/pavnet/16b8fef7-f982-4558-873c-a7fc8d2b7357/"

failedfiles = []

datafiles = [f for f in os.listdir(path) ]
print(f"{len(datafiles)} files found")
Savg = np.zeros(dspconfig["fft_npts"])
iq_arr = [] ; t_arr = []
data_buffer = {"DateTime": []}
for k in dspconfig["vlf_transmitters"].keys():
    data_buffer[k] = []
ti = time.perf_counter()
ii=0
N=len(datafiles)
for f in datafiles:
    pathf = f"{path}/{f}"
    
    try: 
        with open(pathf, "+rb") as file:
            iq = utils.read_binary_IQ(file).T
            t = utils.get_dt_fname_v3(f)
            #print(iq)
        #iq_arr.append(   iq.T )#  to (N,2) shape
        #t_arr.append(t)
        #print(f"File: {f}", end="\r")
        amps_, S = sproc.get_amplitudes(iq)
        #Savg += S
        data_buffer["DateTime"].append(t)
        for name in Txs:
            # appends to queue buffer
            data_buffer[name].append(amps_[name][0])
            # to do: store and/or send to db
        #os.remove(pathf)
    print(ii,ii/N *100, end="\r")
    except ValueError as e:
        failedfiles.append(f)
    #print(f"")
if datafiles: 
    spectrum = Savg / len(datafiles) 
    tf = time.perf_counter()
    print("Nfiles:", len(datafiles),"Avg time spent : ", (tf-ti)/len(datafiles), "s / file  - - - ", end="\r")



85230 files found


KeyboardInterrupt: 

In [8]:
len(data_buffer["DateTime"])

14786

In [10]:
time.perf_counter()-ti

897.8135488370001

In [13]:
plt.plot(data_buffer["DateTime"],data_buffer["NAA"])
plt.show()

In [15]:
import pandas as pd

In [19]:
df = pd.DataFrame(data_buffer, index=data_buffer["DateTime"])

In [20]:
df = df.sort_index()

In [22]:
df.head()

,DateTime,NPM,NAA,NLK,NLM,NAU
2025-05-14 20:01:35,2025-05-14 20:01:35,[465.20265734539015],[5134.357957539795],[1748.9171664360206],[2225.0342575940053],[0.05329164032542714]
2025-05-14 20:01:40,2025-05-14 20:01:40,[334.9045113832787],[5022.121099876195],[1647.5594736221738],[2279.907115489454],[0.052363604197119586]
2025-05-14 20:01:46,2025-05-14 20:01:46,[455.77287813693385],[5139.320852491162],[1695.5928616794745],[2259.574933305622],[0.058843597396608764]
2025-05-14 20:01:50,2025-05-14 20:01:50,[530.0280096209452],[5280.511513670116],[1747.2008370395145],[2359.018841892649],[0.07262251688011079]
2025-05-14 20:01:53,2025-05-14 20:01:53,[433.417534252591],[5170.646023560062],[1698.1251157819902],[2358.4014003886837],[0.053746273301809346]


In [24]:
naa = [x[0] for x in df["NAA"].values]
p

NameError: name 'p' is not defined

In [26]:
plt.plot(naa)

plt.show()

In [27]:
plt.plot(20*np.log10(naa))
plt.show()